# ML bootcamp hackthon challenge: sentiment analysis

At this stage you have learned what machine learning is and you have seen examples of how to use machine learning. Now it is time to see how well YOU can generalize what you have learned to a new challenge.

The hackathon has two tracks: in the **Ideation Track** you can brainstorm on what great apps we can build for the enterprise with this new technology. What are the 'jobs to be done' that machine learning can solve for us in the future? Create an Intrapreneurship-style pitch and present it to the team on Friday. 
https://jam4.sapjam.com/groups/r7ILMAl5MxS8rgHSaNR8L9/overview_page/46399

In the **Data Science Track**, we will run a hackthon challenge on a popular machine learning task: predicting consumer sentiment from online reviews. You are given a set of movie reviews and their labels (positive, negative) and you have to build a system that can predict the sentiment for a new movie review.

We are using the popular polarity data set from Cornell University.
http://www.cs.cornell.edu/people/pabo/movie-review-data/
 

In [ ]:
## download the movie review data set
import sys, os 
import urllib.request

# set http proxy env variable
import os
os.environ['http_proxy'] = 'proxy.sin.sap.corp:8080'

# download file to /tmp
url = "http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz"
file_name = '/tmp/review_polarity.tar.gz'
if not os.path.isfile(file_name):
    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        data = response.read() # a `bytes` object
        out_file.write(data)
    out_file.close()

# make file world readable
os.chmod(file_name, 0o755)

In [ ]:
# extract file to /tmp dir
import tarfile
import os.path


if not os.path.isdir('/tmp/txt_sentoken'):
    os.chdir('/tmp')
    tar = tarfile.open(file_name, "r:gz")
    tar.extractall()
    tar.close()
    

# make files world readable
for label in ['pos', 'neg']:
    os.chmod('/tmp/txt_sentoken/%s/' % label, 0o755)
    for file_name in os.listdir('/tmp/txt_sentoken/%s/' % label):
        os.chmod(os.path.join('/tmp/txt_sentoken/%s/' % label, file_name), 0o755)


In [ ]:
# load positive and negative movie review text

# list to store the examples
reviews = []

# load positive reviews
for file_name in os.listdir('/tmp/txt_sentoken/pos/'):
    with open(os.path.join('/tmp/txt_sentoken/pos/', file_name)) as fin:
        # append sentiment 'pos' label and review text
        reviews.append(('pos', fin.read()))
        
# load negative reviews
for file_name in os.listdir('/tmp/txt_sentoken/neg/'):
    with open(os.path.join('/tmp/txt_sentoken/neg/', file_name)) as fin:
        # append sentiment 'neg' label and review text
        reviews.append(('neg', fin.read()))

# great, there are 2000 reviews in the data set now  
print(len(reviews))

In [ ]:
# randomly split data into 80% training and 20% testing
# each example is a tuple consisting of a (pos/neg)  sentiment label 
# and the text of the review
import random
random.seed(42)
random.shuffle(reviews)

size_train = int(len(reviews)*0.8)
reviews_train = reviews[:size_train]
reviews_test = reviews[size_train:]

print("size of training set:", len(reviews_train))
print("size of test set:", len(reviews_test))

# split data and text
labels_train = [ item[0] for item in reviews_train ]
corpus_train = [ item[1] for item in reviews_train]

labels_test = [ item[0] for item in reviews_test ]
corpus_test = [ item[1] for item in reviews_test]

In [ ]:
# let's look at a few examples of the training set
import pprint
pp = pprint.PrettyPrinter(indent=3)
pp.pprint(reviews_train[:3])

## Level 1: data rookie
Explore the data and find out which words are good features for positive and negative reviews. Visualize the data. Remember what you have learned in the text mining notebook about pre-processing for text. 

In [ ]:
## pre-process, visualize and exploree data

# tokenize the text
import nltk
from nltk import word_tokenize
corpus_train = [ ' '.join(word_tokenize(doc)) for doc in corpus_train ]

# let's check it out
pp.pprint(labels_train[:3])
pp.pprint(corpus_train[:3])

In [ ]:
## clean corpus: remove stop words and lowercase
from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_stopwords(doc):
    # remove stop words and not alphabetic tokens
    return ' '.join((word for word in doc.split() if word.isalpha() and not word in stop))

corpus_train_clean = [ remove_stopwords(doc).lower() for doc in corpus_train ] 
        

In [ ]:
## count word tokens using python Counter
from collections import Counter

# filter out English stop words
cnt = Counter()
cnt_pos = Counter()
cnt_neg = Counter()

for label, doc in zip(labels_train, corpus_train_clean):
    cnt.update(doc.split())
    if label == 'pos':
        cnt_pos.update(doc.split())
    elif label == 'neg':
        cnt_neg.update(doc.split())
    else:
        # should not happen
        print("Warning: unknown label :",  label)
        assert(False)
        
    
# print the 10 most common words
print("Most common words in the corpus")
print(cnt.most_common(10))

# print the 10 most common words in the positive reviews
print("Most common words in the positive reviews")
print(cnt_pos.most_common(10))


# print the 10 most common words in the negative reviews
print("Most common words in the negative reviews")
print(cnt_neg.most_common(10))


## Level 2: data sophomore
Build a first classifier that can predict if a given movie review is positive or negative. 
Remember what you have learned in the data science and text mining notebooks about feature extraction pipelines, training and evaluation.

In [ ]:
## feature extraction, model training on the training set and evaluation on the test set using accuracy


In [ ]:
# create numpy array for labels
import numpy as np

y_train = np.array(list(map(lambda x: +1 if x == 'pos' else -1, labels_train)))
y_test = np.array(list(map(lambda x: +1 if x == 'pos' else -1, labels_test)))


In [ ]:
## baseline: random guess classifier

def random_guess():
    if random.random() > 0.5:
        return +1
    else:
        return -1

# make a random guess 
predicted = [ random_guess() for doc in corpus_test ]

# evalute on the test set
acc = np.mean(predicted == y_test)
print("Accuracy:", acc)

In [ ]:
## naive classifier that predicts +1 if the word 'good' appears in the review

def naive_guess(doc):
    if 'good' in doc.split():
        return +1
    else:
        return -1

# make a naive guess 
predicted = [ naive_guess(doc) for doc in corpus_test ]

# evalute on the test set
acc = np.mean(predicted == y_test)
print("Accuracy:", acc)

In [ ]:
## build 'vanilla' classifier pipeline with all defaults

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('clf', SGDClassifier(random_state=42))
                    ])

# train the model using the pipeline
pipeline.fit(corpus_train, y_train)

# evalute on the test set
predicted = pipeline.predict(corpus_test)
acc = np.mean(predicted == y_test)
print("Accuracy:", acc)

## Level 3: data ninja
Improve your classifier, try out different pre-processing steps, features, classifier models, etc. 
Let's see who can build the most accurate classifier!!
Let's the games begin.

In [ ]:
## build pipelines to test different pre-processing, feaature extraction, and machine learning models
## try to get the best accuracy on the test set


In [ ]:
## use tf-idf

from sklearn.feature_extraction.text import TfidfTransformer

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(random_state=42))
                    ])

# train the model using the pipeline
pipeline.fit(corpus_train, y_train)

# evalute on the test set
predicted = pipeline.predict(corpus_test)
acc = np.mean(predicted == y_test)
print("Accuracy:", acc)

In [ ]:
## use unigrams and bigrams
pipeline = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('clf', SGDClassifier(random_state=42))
                    ])

# train the model using the pipeline
pipeline.fit(corpus_train, y_train)

# evalute on the test set
predicted = pipeline.predict(corpus_test)
acc = np.mean(predicted == y_test)
print("Accuracy:", acc)

In [ ]:
## do grid search over parameter space

from sklearn.grid_search import GridSearchCV

random.seed(42)

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False)
             }

pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(random_state=42))
                    ])

# do grid search
gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)

# train the model using the pipeline
fs_clf = gs_clf.fit(corpus_train, y_train)

best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))


# evalute on the test set
predicted = gs_clf.predict(corpus_test)
acc = np.mean(predicted == y_test)
print("Accuracy:", acc)

In [ ]:
## try out different classifiers

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

random.seed(42)

names = ["SGDClassifier", "Linear SVM", "AdaBoost", "NaiveBayes"]

classifiers = [
    SGDClassifier(),
    SVC(kernel="linear"),
    AdaBoostClassifier(),
    MultinomialNB()]

for name, clf in zip(names, classifiers):
    pipeline = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', clf)
                        ])
    pipeline.fit(corpus_train, y_train)
    # evalute on the test set
    predicted = pipeline.predict(corpus_test)
    acc = np.mean(predicted == y_test)
    print("Model:", name, " Accuracy:", acc)

## (Optional) Level 4: the architect
Build an end-to-end system with REST APIs for sentiment analysis. Input is a movie review, output is the model prediction. You can stay withing python or use the software stack of your trust.

## (Optional) Level 5: the artist
Design a beautiful user experience how the predicted labels of the algorithm can be presented to a consumer on a mobile device. Connect it to the REST webservice or just mock up the inputs.